In [55]:
import json
from authorization import SpotifyAuth

In [56]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [57]:
%autoreload 2

In [58]:
with open('client-secrets-rnr.json','r+') as secrets_file:
    secrets = json.load(secrets_file)

client_id = secrets["client_id"]
client_secret = secrets["client_secret"]
redirect_url = secrets["redirect_url"]

In [59]:
spotify_authorisation = SpotifyAuth(client_id=client_id, client_secret=client_secret, redirect_url=redirect_url)
spotify_authorisation.set_access_token_from_file('access_token.json')

In [62]:
import datetime
yesterday = datetime.datetime.now() - datetime.timedelta(days=1)
# print(yesterday.timestamp()*1e3)
cdc_time = int(yesterday.timestamp()*1e3)

# Get All Songs Played in the last one day

In [60]:
import spotify_functions as spotify

In [105]:
token = spotify_authorisation.get_access_token()
history = spotify.get_listening_history(token, cdc_time)['items']

## EDA over recent history

### Number of songs played

In [106]:
len(history)

6

### Fetch details of a song

#### See available data for a song

In [107]:
history[0].keys()

dict_keys(['track', 'played_at', 'context'])

In [108]:
list(history[0]['track'].keys())

['album',
 'artists',
 'disc_number',
 'duration_ms',
 'explicit',
 'external_ids',
 'external_urls',
 'href',
 'id',
 'is_local',
 'name',
 'popularity',
 'preview_url',
 'track_number',
 'type',
 'uri']

In [109]:
history[0]['played_at']

'2024-08-11T11:03:43.891Z'

In [110]:
print(history[0]['context'])

{'type': 'playlist', 'href': 'https://api.spotify.com/v1/playlists/1TkXrIRsOqiWrVSJSMgk2K', 'external_urls': {'spotify': 'https://open.spotify.com/playlist/1TkXrIRsOqiWrVSJSMgk2K?sessionId=1723334400001&mode=enhanced'}, 'uri': 'spotify:playlist:1TkXrIRsOqiWrVSJSMgk2K'}


# Extract, Transform, Load (ETL)

## Dimensions

### DIM_USER

In [111]:
user = [spotify.get_user_details(spotify_authorisation.get_access_token())]

In [112]:
df_user = pd.DataFrame(user)
df_user['spotify_url'] = df_user['external_urls'].apply(lambda x:x['spotify'])
df_user['profile_pic'] = df_user['images'].apply(get_url, args=[300])
df_user = df_user[['display_name','type','country','product','spotify_url','profile_pic']]
df_user

,display_name,type,country,product,spotify_url,profile_pic
0,sdspot2034,user,IN,premium,https://open.spotify.com/user/sdspot2034,https://i.scdn.co/image/ab6775700000ee8580153a...


#### Load (Type 0 SCD)

In [113]:
metadata = MetaData()

# Define the dimension table schema
dim_table = Table('DIM_USER', metadata,
    Column('user_wid', Integer, primary_key=True, autoincrement=True),
    Column('display_name', String(255), nullable=False, unique=True),
    Column('type', String(255)),
    Column('country', String(255)),
    Column('product', String(255)),
    Column('spotify_url', String(255)),
    Column('profile_pic', String(255))
)

In [114]:
# sql_functions.save_to_sql(df_user, dim_table, key_cols=['spotify_url'], wid=['user_wid'], mode='upsert')
# Code has been commented as this is a Type 0 Dimension with no need for further updates/inserts

### DIM_ARTIST

In [115]:
import pandas as pd
from sqlalchemy import Table, MetaData, Column, Integer, String
import sql_functions

In [116]:
# stacked list-comprehension
artist_list = [artist for artists in [play['track']['artists'] for play in history] for artist in artists]

In [117]:
df_artists = pd.DataFrame(artist_list)
df_artists = df_artists.drop(columns=['href','uri','external_urls'])
df_artists = df_artists.rename(columns={'id':'spotify_id','name':'artist_name','type':'artist_type'})
df_artists.drop_duplicates(inplace=True)
# df_artists

#### Load (Type 1 SCD)

In [118]:
metadata = MetaData()

# Define the dimension table schema
dim_table = Table('DIM_ARTIST', metadata,
    Column('artist_wid', Integer, primary_key=True, autoincrement=True),
    Column('spotify_id', String(255), nullable=False, unique=True),
    Column('artist_name', String(255)),
    Column('artist_type', String(255))
)

In [119]:
sql_functions.save_to_sql(df_artists, dim_table, key_cols=['spotify_id'], wid=['artist_wid'], mode='upsert')

### DIM_ALBUM

In [120]:
album_list = [play['track']['album'] for play in history]

In [121]:
def get_url(x,height=640):
    for elem in x:
        if elem['height'] == height: return elem['url']
    return None

In [122]:
df_album = pd.DataFrame(album_list)
df_album['cover_image_url'] = df_album['images'].apply(get_url)
df_album = df_album.drop(columns=['href','uri','external_urls', 'images','artists','type'])
df_album = df_album.drop_duplicates()
df_album = df_album.rename(columns={'id':'album_id','name':'album_name'})
# df_album

#### Load (Type 1 SCD)

In [123]:
from sqlalchemy import MetaData, Table, Column, Integer, String, Text, Date

metadata = MetaData()

dim_album = Table(
    'DIM_ALBUM', metadata,
    Column('album_wid', Integer, primary_key=True, autoincrement=True),
    Column('album_id', String(62), nullable=False, unique=True),
    Column('album_type', String(20), nullable=False),
    Column('album_name', Text, nullable=False),
    Column('release_date', Date),
    Column('release_date_precision', String(20)),
    Column('total_tracks', Integer),
    Column('cover_image_url', Text)
)

In [124]:
sql_functions.save_to_sql(df_album, dim_album, key_cols = ['album_id'], wid = ['album_wid'], mode = 'upsert')

### BRIDGE_ALBUM_ARTIST

In [125]:
dim_artist = sql_functions.read_from_sql('DIM_ARTIST')
dim_album = sql_functions.read_from_sql('DIM_ALBUM')
df_album = pd.DataFrame(album_list)
df_album_map = df_album[['id','artists']]
df_album_map = df_album_map.explode('artists')
df_album_map['artist_id'] = df_album_map['artists'].apply(lambda x:x['id'])
df_album_map = df_album_map.drop(columns=['artists'])
# df_album_map

In [126]:
# Join with DIM_ARTIST, DIM_ALBUM to get ARTIST_WID, ALBUM_WID
df_bridge_album_artist = df_album_map.merge(dim_artist, left_on='artist_id', right_on='spotify_id')
df_bridge_album_artist = df_bridge_album_artist.merge(dim_album, left_on='id', right_on='album_id')
df_bridge_album_artist = df_bridge_album_artist[['album_wid','artist_wid']]
df_bridge_album_artist = df_bridge_album_artist.drop_duplicates()
# df_bridge_album_artist

#### Load (Type 1 SCD)

In [127]:
from sqlalchemy import MetaData, Table, Column, Integer

metadata = MetaData()

bridge_album_artist = Table(
    'BRIDGE_ALBUM_ARTIST', metadata,
    Column('album_artist_map_wid', Integer, primary_key=True, autoincrement=True),
    Column('album_wid', Integer, nullable=False),
    Column('artist_wid', Integer, nullable=False)
)

In [128]:
sql_functions.save_to_sql(
    df_bridge_album_artist
    , bridge_album_artist
    , key_cols = ['album_wid', 'artist_wid']
    , wid = ['album_artist_map_wid']
    , mode = 'ignore'
)

### DIM_SONG

In [129]:
dim_album = sql_functions.read_from_sql('DIM_ALBUM')

In [130]:
history[0]['track'].keys()

dict_keys(['album', 'artists', 'disc_number', 'duration_ms', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track_number', 'type', 'uri'])

In [131]:
track_list = [play['track'] for play in history]

In [132]:
df_songs = pd.DataFrame(track_list)
df_songs = df_songs.explode('artists')
df_songs['artist_id'] = df_songs['artists'].apply(lambda x: x['id'])
df_songs['isrc_id'] = df_songs['external_ids'].apply(lambda x: x['isrc'])
df_songs = df_songs.drop(columns=['href','uri','external_urls','artists','external_ids', 'artist_id', 'popularity'])
df_songs['album_id'] = df_songs['album'].apply(lambda x:x['id'])
df_songs = df_songs.merge(dim_album[['album_wid','album_id']], left_on='album_id', right_on='album_id')
df_songs = df_songs.drop(columns=['album','album_id'])
df_songs = df_songs.rename(columns={'id':'song_id'})
df_songs = df_songs.drop_duplicates()
df_songs

,disc_number,duration_ms,explicit,song_id,is_local,name,preview_url,track_number,type,isrc_id,album_wid
0,1,149706,False,6XQHlsNu6so4PdglFkJQRJ,False,"City Of Stars - From ""La La Land"" Soundtrack",https://p.scdn.co/mp3-preview/fde76ee19ffd4123...,9,track,USUG11600656,71
2,1,235813,False,72R0X0h8YaxYNpegeoOl0M,False,Kiss Me Slowly,https://p.scdn.co/mp3-preview/6de18c73982456bc...,7,track,USUM71100714,72
4,1,206218,False,4ENjopIjOD5PKPb52NKNKo,False,Hero,https://p.scdn.co/mp3-preview/fea431c768d0d137...,1,track,QZXDB2300087,60
5,1,87000,False,6me7F0aaZjwDo6RJ5MrfBD,False,Evergreen,https://p.scdn.co/mp3-preview/fca7555fba8261b7...,1,track,SEYOK1732035,47
6,1,252818,False,456WNXWhDwYOSf5SpTuqxd,False,Dog Days Are Over,https://p.scdn.co/mp3-preview/f2e9f824fbfb697b...,1,track,GBUM70900209,66


#### Load (Type 1 SCD)

In [133]:
from sqlalchemy import BigInteger, Boolean, Text


metadata = MetaData()

dim_song = Table(
    'DIM_SONG', metadata,
    Column('song_wid', Integer, primary_key=True, autoincrement=True),
    Column('song_id', String(62), nullable=False, unique=True),
    Column('album_wid', Integer),
    Column('disc_number', Integer),
    Column('duration_ms', BigInteger, nullable=False),
    Column('explicit', Boolean),
    Column('is_local', Boolean),
    Column('name', Text, nullable=False),
    Column('preview_url', Text),
    Column('track_number', Integer),
    Column('type', String(50)),
    Column('isrc_id', String(100))
)

In [134]:
sql_functions.save_to_sql(df_songs, dim_song, key_cols = ['song_id'], wid = ['song_wid'], mode = 'upsert')

### FACT_PLAY

In [144]:
dim_song = sql_functions.read_from_sql('DIM_SONG')
df_play = pd.DataFrame(history)
df_play['song_id'] = df_play['track'].apply(lambda x:x['id'])
df_play = df_play.merge(dim_song, 'left', on = 'song_id')
df_play['context_playlist'] = df_play['context'].apply(lambda x:x['href'] if x and 'href' in x.keys() else None)
df_play = df_play[['song_wid','played_at','context_playlist']]
df_play

,song_wid,played_at,context_playlist
0,89,2024-08-11T11:03:43.891Z,https://api.spotify.com/v1/playlists/1TkXrIRsO...
1,90,2024-08-11T11:01:13.712Z,https://api.spotify.com/v1/playlists/1TkXrIRsO...
2,90,2024-08-11T11:00:54.486Z,https://api.spotify.com/v1/playlists/1TkXrIRsO...
3,50,2024-08-11T10:43:44.672Z,None
4,36,2024-08-11T06:42:53.419Z,None
5,87,2024-08-11T06:33:50.717Z,None
